In [ ]:
#   This is filename:  Uhlig_solver_abreviated.ipynb
#   It is an abreviated translation of Uhlig's 1997 solve.m
#   code into Python. 

#  The inputs into this notebook are the matrices
#   AA, BB, CC, DD, FF, GG, HH, JJ, KK, LL, MM, NN

#   The output of this notebook are the matices 
#   PP,  QQ,  RR,  SS  and  WW.   
#   All the internediate matrices are available too and 
#   can easily be eyeballed by the user by deleting 
#   the " # "  tags which precede print related commands.


#  The solution method is Harald Uhlig's, as he 
#   described in
#     " A Tooklit for Analyzing Nonlinear Dynamic
#      Stochastic Models Easily "  Discussion Paper 101
#  Federal Reserve Bank of Minneapolis  June 1995
#  I use his convention in naming matrices, specifically 
#
#  0 = AA x_(t) +  BB x_(t-1)  +  CC y_(t)  +  DD z_(t)
#
#  0 = E_(t) { FF x_(t+1) +  GG x_(t)  +  HH x_(t-1) +  JJ y_(t+1) +  KK y_(t) +  LL z_(t+1) +  MM z_(t) }
#
#  E_(t) { z_(t+1) } = NN z_(t)  +  E_(t) { e_(t+1) }
#
#  The recursive equililbrium laws of motion are specified as
#
#   x_(t) = PP x_(t-1) + QQ z_(t)
#
#   y_(t) = RR x_(t-1) + SS z_(t)
#
#  z_(t) = NN z_(t-1)  +  e_(t)
#
#  And for convenience
#
#  [ x(t)', y(t)', z(t)' ] = WW [ x(t)', z(t)' ] 
#




import numpy as np
from numpy import hstack, vstack, zeros, dot, eye, kron

import scipy as sp
from scipy import linalg as la
from scipy.linalg import null_space
import scipy.optimize as opt

import matplotlib.pyplot as pl


np.set_printoptions(precision=4, suppress=True)



AA = np.matrix(AA)
BB = np.matrix(BB)
CC = np.matrix(CC)
DD = np.matrix(DD)
FF = np.matrix(FF)
GG = np.matrix(GG)
HH = np.matrix(HH)
JJ = np.matrix(JJ)
KK = np.matrix(KK)
LL = np.matrix(LL)
MM = np.matrix(MM)
NN = np.matrix(NN)
Sigma =  np.matrix(Sigma_EPS)


m_states = FF.shape[1]
l_equ = CC.shape[0]
n_endog = CC.shape[1]
k_exog  = min(NN.shape)



#print('CC shape =', CC.shape)
#print('CC dim =', np.ndim(CC))
#print('CC rank =', np.linalg.matrix_rank(CC))
#print('Rank of CC needs to be = > n_endog')
#print('n_endog =', n_endog)
#print(' ')

CC_plus =  np.linalg.pinv(CC)
CC_plus = np.matrix(CC_plus)
#print('CC_plus= ', CC_plus)
#print('CC_plus shape =', CC_plus.shape)
#print('CC_plus dim =', np.ndim(CC_plus))
#print('CC_plus rank =', np.linalg.matrix_rank(CC_plus))
#print(' ')

CC_T = np.transpose(CC)
#print('CC_T shape =', CC_T.shape)
#print('CC_T dim =', np.ndim(CC_T))
#print('CC_T rank =', np.linalg.matrix_rank(CC_T))
#print(' ')

CC_null = sp.linalg.null_space(CC_T)
#print('CC_null shape =', CC_null.shape)
#print('CC_null dim =', np.ndim(CC_null))
#print('CC_null rank =', np.linalg.matrix_rank(CC_null))
#print(' ')

CC_0 =  np.transpose(CC_null)
#print('CC_0 shape =', CC_0.shape)
#print('CC_0 dim =', np.ndim(CC_0))
#print('CC_0 rank =', np.linalg.matrix_rank(CC_0))
#print(' ')

Gamma_mat = np.vstack((dot(CC_0,AA),
                                         dot(dot(JJ,CC_plus),BB) - GG + dot(dot(KK,CC_plus),AA)))
#print('Gamma_mat = ', Gamma_mat)
#print('Gamma_mat shape =', Gamma_mat.shape)
#print('Gamma_mat dim =', np.ndim(Gamma_mat))
#print('Gamma_mat rank =', np.linalg.matrix_rank(Gamma_mat))
#print(' ')

Psi_mat = np.vstack((zeros((l_equ-n_endog)*m_states).reshape(l_equ-n_endog,m_states),
                                FF - dot(dot(JJ,CC_plus),AA)))
#print('Psi_mat = ', Psi_mat)
#print('Psi_mat shape =', Psi_mat.shape)
#print('Psi_mat dim =', np.ndim(Psi_mat))
#print('Psi_mat rank =', np.linalg.matrix_rank(Psi_mat))
#print(' ')

Theta_mat = np.vstack((dot(CC_0,BB),
                                       dot(dot(KK,CC_plus),BB) - HH))
#print('Theta_mat = ', Theta_mat)
#print('Theta_mat shape =', Theta_mat.shape)
#print('Theta_mat dim =', np.ndim(Theta_mat))
#print('Theta_mat rank =', np.linalg.matrix_rank(Theta_mat))  
#print(' ')

Xi_mat = np.vstack((np.hstack((Gamma_mat, Theta_mat)),
                                 np.hstack((eye(m_states),zeros(m_states**2).reshape(m_states,m_states)))))
#print('Xi_mat = ', Xi_mat)
#print('Xi_mat shape =', Xi_mat.shape)
#print('Xi_mat dim =', np.ndim(Xi_mat))
#print('Xi_mat rank =', np.linalg.matrix_rank(Xi_mat))  
#print(np.ndim(eye(m_states)))
#print((eye(m_states)).shape)
#print((zeros(m_states**2).reshape(m_states,m_states)).shape)
#print(' ')

Delta_mat = np.vstack((np.hstack((Psi_mat, zeros(m_states**2).reshape(m_states,m_states))),
                    np.hstack((zeros(m_states**2).reshape(m_states,m_states),eye(m_states)))))
#print('Delta_mat = ', Delta_mat)
#print('Delta_mat shape =', Delta_mat.shape)
#print('Delta_mat dim =', np.ndim(Delta_mat))
#print('Delta_mat rank =', np.linalg.matrix_rank(Delta_mat))
#print(' ')

eVals, eVecs = la.eig(Xi_mat, Delta_mat)
#eVals_diag = np.diag(eVals)
#eVals = eVals_diag
#print('eVals = ', eVals)
#print('eVals shape =', eVals.shape)
#print('eVals =', np.ndim(eVals))
#print('eVals rank =', np.linalg.matrix_rank(eVals))  
#print(' ')
#print('eVecs = ', eVecs)
#print('eVecs dim =', np.ndim(eVecs))
#print('eVecs shape =', eVecs.shape)
#print(' ')
#print('eVecs rank =', np.linalg.matrix_rank(eVecs))    
#print('m_states = ', m_states)
#print('rank of eVecs should be = > m_states ')
#print(' ')

Xi_sortabs = np.sort(abs(eVals))
#print('Xi_sortabs = ', Xi_sortabs)
#print('Xi_sortabs shape =', Xi_sortabs.shape)
#print(' ')
Xi_sortindex = np.argsort(abs(eVals))
#print('Xi_sortindex = ', Xi_sortindex)
#print('Xi_sortindex shape =', Xi_sortindex.shape)
#print(' ')
Xi_sortVec = np.array([eVecs[:, i] for i in Xi_sortindex]).T
#print('Xi_sortVec = ', Xi_sortVec)
#print('Xi_sortVec shape =', Xi_sortVec.shape)
#print(' ')
Xi_sortval = eVals[Xi_sortindex]
#print('Xi_sortval = ', Xi_sortval)
#print('Xi_sortval =', Xi_sortval.shape)
#print(' ')
Xi_select = np.arange(0, m_states)
#print('Xi_select = ', Xi_select)
#print('Xi_select =', Xi_select.shape)
#print(' ')
#  Note that the arguments of Xi_sortvec and Xi_sortval
#  can be replaced with manually chosen roots Xi_manual
#print('Xi_sortVec = ', Xi_sortVec)
#print('Xi_sortVec shape =', Xi_sortVec.shape)
#print('Xi_sortVec dim =', np.ndim(Xi_sortVec))
#print('Xi_sortVec rank =', np.linalg.matrix_rank(Xi_sortVec))  
#print('Xi_sortVec max =', np.max(np.absolute(Xi_sortVec)))
#print(' ')
#print('Xi_sortval = ', Xi_sortval)
#print(' ')
#Xi_sortval_imag = Xi_sortval.imag
#print('Xi_sortval[Xi_select].imag = ', Xi_sortval[Xi_select].imag)
#print('Does Xi_sortval[Xi_select] have complex numbers?')
#print('Go check your parameterization.')
#print('  ')
#print('Xi_sortval[Xi_select].real = ', Xi_sortval[Xi_select].real)
#print('  ')
#print('abs(Xi_sortval[Xi_select]) max = ', np.max(abs(Xi_sortval[Xi_select])), ' if > than 1 then have unstable roots')
#print('  ')
#print('Xi_sortabs[Xi_select] max = ', np.max(Xi_sortabs[np.max(Xi_select)]), ' should be less than 1')
#print('  ')


##Lambda_mat = np.diag(Xi_sortval[Xi_select].real)
Lambda_mat = np.diag(Xi_sortval[Xi_select])
#print('Xi_select = ', Xi_select)
#print('  ')
#print('Lambda_mat = ', Lambda_mat)
#print('Lambda_mat shape =', Lambda_mat.shape)
#print('Lambda_mat dim =', np.ndim(Lambda_mat))
#print('Lambda_mat rank =', np.linalg.matrix_rank(Lambda_mat))  
#print(' ')


##Omega_mat = Xi_sortVec[m_states:2 * m_states, Xi_select].real
Omega_mat = Xi_sortVec[m_states:2 * m_states, Xi_select]
#print('Omega_mat = ', Omega_mat)
#print('Omega_mat shape =', Omega_mat.shape)
#print('Omega_mat =', np.ndim(Omega_mat))
#print('Omega_mat rank =', np.linalg.matrix_rank(Omega_mat)) 
#print('rank should be = ', m_states)
#print(' ')

##PP = dot(dot(Omega_mat, Lambda_mat), la.pinv(Omega_mat)).real
PP = dot(dot(Omega_mat, Lambda_mat), la.pinv(Omega_mat))
PP_imag = np.imag(PP)
PP = np.real(PP)
#print('PP = ', PP)
#print('PP =', PP.shape)
#print('PP dim =', np.ndim(PP))
#print('PP rank =', np.linalg.matrix_rank(PP))
#print(' ')

RR = - dot(CC_plus, (dot(AA, PP) + BB))
#print('RR = ', RR)
#print('RR =', RR.shape)
#print('RR dim =', np.ndim(RR))
#print('RR rank =', np.linalg.matrix_rank(RR))
#print(' ')

VV = np.vstack((hstack((kron(eye(k_exog), AA), \
                           kron(eye(k_exog), CC))), hstack((kron(NN.T, FF) +\
                           kron(eye(k_exog), dot(FF, PP) + dot(JJ, RR) + GG),\
                           kron(NN.T, JJ) + kron(eye(k_exog), KK)))))
#rank_count = k_exog*(m_states+n_endog)
#print('VV = ', VV)
#print('VV =', VV.shape)
#print('VV dim =', np.ndim(VV))
#print('VV rank =', np.linalg.matrix_rank(VV))
#print('rank should be = ',rank_count)
#print(' ')

LLNN_plus_MM = dot(LL,NN) + MM
#print('LLNN_plus_MM = ', LLNN_plus_MM)
#print('LLNN_plus_MM shape =', LLNN_plus_MM.shape)
#print('LLNN_plus_MM dim =', np.ndim(LLNN_plus_MM))
#print('LLNN_plus_MM rank =', np.linalg.matrix_rank(LLNN_plus_MM))
#print('   ')

DD_flat = DD.reshape((DD.shape[0])*(DD.shape[1]),1)
#print('DD_flat = ', DD_flat)
#print('DD_flat shape=', DD_flat.shape) 
#print('DD_flat dim =', np.ndim(DD_flat)) 
#print('DD_flat rank =', np.linalg.matrix_rank(DD_flat))
#print('  ')

LLNN_plus_MM_flat = LLNN_plus_MM.reshape((LLNN_plus_MM.shape[0])*(LLNN_plus_MM.shape[1]),1)
#print('LLNN_plus_MM_flat', LLNN_plus_MM_flat)
#print('LLNN_plus_MM_flat shape =', LLNN_plus_MM_flat.shape) 
#print('LLNN_plus_MM_flat dim =', np.ndim(LLNN_plus_MM_flat)) 
#print('LLNN_plus_MM_flat =', np.linalg.matrix_rank(LLNN_plus_MM_flat))
#print('  ')

denominator = np.vstack((DD_flat,LLNN_plus_MM_flat))
#print('denominator = ',denominator)
#print('denominator shape =', denominator.shape) 
#print('denominator dim =', np.ndim(denominator)) 
#print('denominator=', np.linalg.matrix_rank(denominator))
#print('  ')

QQSS_vec = - la.solve(VV,denominator) 
#print('QQSS_vec = ', QQSS_vec) 
#print('QQSS_vec =', QQSS_vec.shape) 
#print('QQSS_vec dim =', np.ndim(QQSS_vec)) 
#print('QQSS_vec rank =', np.linalg.matrix_rank(QQSS_vec))
#print('QQSS_vec max element = ', np.max(np.absolute(QQSS_vec)))
#print('This should not be infinite in value.')
#print('  ')

QQ = np.reshape(np.matrix(QQSS_vec[0:m_states * k_exog , 0]), (m_states, k_exog), 'F')
#print('QQ = ', QQ)
#print('QQ =', QQ.shape) 
#print('QQ dim =', np.ndim(QQ)) 
#print('QQ rank =', np.linalg.matrix_rank(QQ))
#print('  ')

SS = np.reshape(QQSS_vec[(m_states * k_exog ):((m_states + n_endog) * k_exog), 0], (n_endog, k_exog), 'F')
#print('SS = ', SS)
#print('SS =', SS.shape) 
#print('SS dim =', np.ndim(SS)) 
#print('SS rank =', np.linalg.matrix_rank(SS))
#print('  ')

WW = np.vstack((np.hstack((eye(m_states), zeros((m_states, k_exog)))),
                             np.hstack((dot(RR, np.linalg.pinv(PP)), (SS - dot(dot(RR, np.linalg.pinv(PP)), QQ)))),
                             np.hstack((zeros((k_exog, m_states)), eye(k_exog)))))
#print('WW = ', WW)
#print('WW =', WW.shape) 
#print('WW dim =', np.ndim(WW)) 
#print('WW rank =', np.linalg.matrix_rank(WW))
#print('  ')

print('Done. You have the matrices PP QQ  RR  SS  WW.')
